# RAG in LangChain

The jupyter notebook of storing the code and result of the Medium article

In [1]:
from pathlib import Path
from langchain_community.llms import Ollama
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader                                                                                                                                                                                                                                                                                                                                                                                                                                      
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain

file_dir_path = Path('cyberpunk_2077_phantom_liberty')

llm = Ollama(model="llama2:7b-chat-q4_K_M")
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

loader = DirectoryLoader(str(file_dir_path), glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()                                                              

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# create the vector database
vector_db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_function,
)

retriever = vector_db.as_retriever()

# make a chain

# create the chain to answer questions
# with RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm, 
                                    chain_type="stuff", 
                                    retriever=retriever, 
                                    return_source_documents=True)

# with RetrievalQAWithSourcesChain
chain02 = RetrievalQAWithSourcesChain.from_chain_type(llm=llm,
                                                        chain_type="stuff",
                                                        retriever=retriever,
                                                        return_source_documents=True)

# full example
prompt = 'What are the drawbacks of the game?'
llm_response = chain(prompt)
llm_response02 = chain02(prompt)

# print the response
print(llm_response)

# cleanup
# uncomment if finished testing
vector_db.delete_collection()

/root/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/miniforge3/envs/fyp-test-wsl-tm/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Created a chunk of size 1087, which is longer than the specified 1000
Created a chunk of size 1208, which is longer than the specified 1000
/root/miniforge3/envs/fyp-test-wsl-tm/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and

{'query': 'What are the drawbacks of the game?', 'result': "I don't know the answer to your question as the text you provided does not mention any drawbacks or negative aspects of the game. In fact, it seems to highlight the expansion's strengths and how it improves upon the original game. The text mentions that the new cyberware system adds another layer of complexity to creating an effective build, and that the Relic skill tree is not particularly deep but adds a wrinkle to the formula with new cyberware and perks. It also highlights the expansion's ability to provide three completely different gameplay experiences, which is a significant strength. Therefore, I cannot answer your question as there are no drawbacks mentioned in the text you provided.", 'source_documents': [Document(page_content="The new cyberware system adds another layer to creating an effective build, letting you fine-tune your character with specific stat buffs, additional abilities, and quickhacks. Phantom Liberty

In [2]:
print(documents)

[Document(page_content='Given Phantom Liberty\'s superspy theme, I expected, nay, demanded a good high-society shindig infiltration level, and Cyberpunk 2077\'s first and only expansion pack delivered. \n\nThe centerpiece of the soiree mission was a long, tense dialogue puzzle where I had to earn the trust of a pair of deliciously awful French twins while playing high stakes roulette. I had to engage, provoke, and eventually befriend the international con artists all while I watched my FIA (Cyberpunk future CIA)-provided funds dwindle away\u2060. Shoulda bet on black. At the last minute I chose a particularly inflammatory and insulting dialogue option and my handler pre-emptively chewed me out over the radio, but the little freaks absolutely ate it up. Maybe I am cut out for this superspy stuff.\n\nPhantom Liberty is an extra-refined bite of Cyberpunk 2077—an expansion pack\'s expansion pack. It doesn\'t reinvent the game as a whole, but it\'s a fantastic final outing for V and Night C

In [3]:
print(chain)

combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=Ollama(model='llama2:7b-chat-q4_K_M')), document_variable_name='context') return_source_documents=True retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7fd26109efa0>)


In [4]:
print(llm_response02)

{'question': 'What are the drawbacks of the game?', 'answer': 'The president did not mention Michael Jackson in any of the provided sources.', 'sources': '', 'source_documents': [Document(page_content="The new cyberware system adds another layer to creating an effective build, letting you fine-tune your character with specific stat buffs, additional abilities, and quickhacks. Phantom Liberty introduces the Relic skill tree as well, and while it isn't particularly deep, it adds a wrinkle to the formula with new cyberware like the quickhack-infused Monowire melee weapon, or additional perks that give an upper hand for stealth or hitting enemy weak spots. Cyberpunk 2077's RPG identity is much stronger now than it was previously and Phantom Liberty's combat scenarios substantially benefit from this reinvented foundation. Together, it all results in a uniquely satisfying gameplay experience.", metadata={'source': 'cyberpunk_2077_phantom_liberty/cyberpunk_2077_phantom_liberty_02.txt'}), Docu

In [5]:
print(chain02)

combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\nQUESTION: Which state/country\'s law governs the interpretation of the contract?\n=========\nContent: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.\nSource: 28-pl\nContent: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)

The prompt template is of RetrievalQAWithSourceChain is from [here](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/qa_with_sources/map_reduce_prompt.py#L52), which can be traced from the base class of StuffDocumentsChain: [Github](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/qa_with_sources/base.py#L49)

However, one can observe that the few-shot prompting technique is not helpful to our prompt, and therefore, we are only using RetrievalQA.from_chain_type() in this article

---

## Different TextSplitter

Different splitters: [LangChain](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/)

Visualize splitters: [Chunkviz utility](https://chunkviz.up.railway.app/) created by Greg Kamradt

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# load a single document and split it into chunks
doc_path = Path("cyberpunk_2077_phantom_liberty/cyberpunk_2077_phantom_liberty_01.txt")

loader = TextLoader(str(doc_path))
documents = loader.load()

In [4]:
# character text splitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)
docs

[Document(page_content="Given Phantom Liberty's superspy theme, I expected, nay, demanded a good high-society shindig infiltration level, and Cyberpunk 2077's first and only expansion pack delivered. \n\nThe centerpiece of the soiree mission was a long, tense dialogue puzzle where I had to earn the trust of a pair of deliciously awful French twins while playing high stakes roulette. I had to engage, provoke, and eventually befriend the international con artists all while I watched my FIA (Cyberpunk future CIA)-provided funds dwindle away\u2060. Shoulda bet on black. At the last minute I chose a particularly inflammatory and insulting dialogue option and my handler pre-emptively chewed me out over the radio, but the little freaks absolutely ate it up. Maybe I am cut out for this superspy stuff.", metadata={'source': 'cyberpunk_2077_phantom_liberty/cyberpunk_2077_phantom_liberty_01.txt'}),
 Document(page_content='Phantom Liberty is an extra-refined bite of Cyberpunk 2077—an expansion pac

In [5]:
# recursive character text splitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600, chunk_overlap=100
)
docs = text_splitter.split_documents(documents)
docs

[Document(page_content="Given Phantom Liberty's superspy theme, I expected, nay, demanded a good high-society shindig infiltration level, and Cyberpunk 2077's first and only expansion pack delivered.", metadata={'source': 'cyberpunk_2077_phantom_liberty/cyberpunk_2077_phantom_liberty_01.txt'}),
 Document(page_content='The centerpiece of the soiree mission was a long, tense dialogue puzzle where I had to earn the trust of a pair of deliciously awful French twins while playing high stakes roulette. I had to engage, provoke, and eventually befriend the international con artists all while I watched my FIA (Cyberpunk future CIA)-provided funds dwindle away\u2060. Shoulda bet on black. At the last minute I chose a particularly inflammatory and insulting dialogue option and my handler pre-emptively chewed me out over the radio, but the little freaks absolutely ate it up. Maybe I am cut out for this superspy stuff.', metadata={'source': 'cyberpunk_2077_phantom_liberty/cyberpunk_2077_phantom_li

In [6]:
# using token length instead of character length in RecursiveCharacterTextSplitter

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
embedding_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

def custom_len_func(text:str) -> int:
  return len(tokenizer.encode(text, add_special_tokens=False))

# as mentioned in the Github Issue, different splitters have implemented .from_huggingface_tokenizer() and .from_tiktoken_encoder() methods
text_spliter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
  tokenizer=tokenizer,
  chunk_size=250,
  chunk_overlap=40,
)

docs = text_spliter.split_documents(documents)
docs

[Document(page_content="Given Phantom Liberty's superspy theme, I expected, nay, demanded a good high-society shindig infiltration level, and Cyberpunk 2077's first and only expansion pack delivered. \n\nThe centerpiece of the soiree mission was a long, tense dialogue puzzle where I had to earn the trust of a pair of deliciously awful French twins while playing high stakes roulette. I had to engage, provoke, and eventually befriend the international con artists all while I watched my FIA (Cyberpunk future CIA)-provided funds dwindle away\u2060. Shoulda bet on black. At the last minute I chose a particularly inflammatory and insulting dialogue option and my handler pre-emptively chewed me out over the radio, but the little freaks absolutely ate it up. Maybe I am cut out for this superspy stuff.", metadata={'source': 'cyberpunk_2077_phantom_liberty/cyberpunk_2077_phantom_liberty_01.txt'}),
 Document(page_content='Phantom Liberty is an extra-refined bite of Cyberpunk 2077—an expansion pac

---

## Creating vector database with Chroma

In [7]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# create the vector database
vector_db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_function,
)

---

## Testing the retriever

In [9]:
# adjust the number of returned chunks by passing a dictionary like with key = 'k' and value as an positive int
retriever = vector_db.as_retriever(search_kwargs={"k": 5})

rag_question = "How is the gameplay?"

# or pass the number of returned chunks like below
retrieved_docs = retriever.get_relevant_documents(query=rag_question, k=5)

In [10]:
retrieved_docs

[Document(page_content='Phantom Liberty is an extra-refined bite of Cyberpunk 2077—an expansion pack\'s expansion pack. It doesn\'t reinvent the game as a whole, but it\'s a fantastic final outing for V and Night City, as well as one of the best individual stories CD Projekt Red has told to date. Meanwhile, Cyberpunk 2077\'s 2.0 update, free to all players, has reforged the original looter shooter gear deluge and "+5% poison resistance"-style perks system, giving us a more solid RPG whose under-the-hood systems better compliment its shooting, slashing, and stealthing.\n\nAfter a legendarily fraught launch and three years of rebuilding, Cyberpunk 2077 is finally the RPG its world and characters deserved, and Phantom Liberty is a fine last hurrah for a game that\'s taken CD Projekt so long to hone.', metadata={'source': 'cyberpunk_2077_phantom_liberty/cyberpunk_2077_phantom_liberty_01.txt'}),
 Document(page_content="Still, I'm extremely pleased with Phantom Liberty\u2060—CD Projekt can h

---

## Prompting

Please go back to the first cell and read the printouts of `llm_response ` and `chainXX` variables.

---

## Customizing the chain

In [18]:
from pathlib import Path
from langchain_community.llms import Ollama
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader                                                                                                                                                                                                                                                                                                                                                                                                                                      
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain_core.prompts import PromptTemplate

file_dir_path = Path('cyberpunk_2077_phantom_liberty')

llm = Ollama(model="llama2:7b-chat-q4_K_M")
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

loader = DirectoryLoader(str(file_dir_path), glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()                                                              

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# create the vector database
vector_db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_function,
)

retriever = vector_db.as_retriever()

# make a chain

# create the chain to answer questions
# with RetrievalQA
# chain = RetrievalQA.from_chain_type(llm=llm, 
#                                     chain_type="stuff", 
#                                     retriever=retriever, 
#                                     return_source_documents=True)

prompt_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question. 

{context}

Question: {question}

If you don't know the answer, just output a json with all values in the json as 'NA'. Do NOT try to make up an answer.
Only output the JSON. Do NOT output other text.'''

my_question = \
'''Extract the following aspects of the game from the reviews. Output a json with each of the aspects as key, and the extracted information as the value. The format of the json is {"ASPECT":"INFORMATION"}. The aspects are: [Gameplay, Graphics, Sound, Performance, Bug, Suggestion, Price, Overall]
'''

chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=retriever,
                                    chain_type_kwargs={
                                        "prompt": PromptTemplate(
                                            template=prompt_template,
                                            input_variables=["context", "question"],
                                            ),
                                        },
                                    return_source_documents=True
                                    )

# full example

llm_response = chain.invoke({'question':my_question, 'query': my_question})
# llm_response02 = chain02(prompt)

# print the response
print(llm_response)

# cleanup
vector_db.delete_collection()

Created a chunk of size 1087, which is longer than the specified 1000
Created a chunk of size 1208, which is longer than the specified 1000


{'question': 'Extract the following aspects of the game from the reviews. Output a json with each of the aspects as key, and the extracted information as the value. The format of the json is {"ASPECT":"INFORMATION"}. The aspects are: [Gameplay, Graphics, Sound, Performance, Bug, Suggestion, Price, Overall]\n', 'query': 'Extract the following aspects of the game from the reviews. Output a json with each of the aspects as key, and the extracted information as the value. The format of the json is {"ASPECT":"INFORMATION"}. The aspects are: [Gameplay, Graphics, Sound, Performance, Bug, Suggestion, Price, Overall]\n', 'result': '{\n"Gameplay": "Excellently written story • World-class acting, not only by Idris Elba and Keanu Reeves • Graphically stunning – Cyberpunk 2077 is the new Crysis • Skill tree conversion trimmed for RPG • Dogtown is Narcos’ Medellín by day, Blade Runner by night • Stealth Bond, John Wick, or Mr. Robot – each game type feels unique • Has a lot more Keanu Reeves than we

In [19]:
print(llm_response['result'])

{
"Gameplay": "Excellently written story • World-class acting, not only by Idris Elba and Keanu Reeves • Graphically stunning – Cyberpunk 2077 is the new Crysis • Skill tree conversion trimmed for RPG • Dogtown is Narcos’ Medellín by day, Blade Runner by night • Stealth Bond, John Wick, or Mr. Robot – each game type feels unique • Has a lot more Keanu Reeves than we were expecting",
"Graphics": "Graphically stunning – Cyberpunk 2077 is the new Crysis",
"Sound": "Expertly written story",
"Performance": "Rebuilt RPG elements make combat scenarios truly satisfying regardless of playstyle • Remarkable detail and density give Dogtown a brutal, complex identity",
"Bug": "NA",
"Suggestion": "NA",
"Price": "NA",
"Overall": "Phantom Liberty is CD Projekt RED’s masterpiece. Not only is Cyberpunk 2077 Phantom Liberty graphically easily three generations ahead of the entire industry and redefines how we experience video games with pathtracing, it’s also written even more thrillingly and staged eve

In [20]:
print(llm_response['source_documents'])

[Document(page_content='The Good\nThrilling main story with two stunning yet wildly different routes and conclusions\nGreat performances make new and returning characters the heart of Cyberpunk\nNew ending for the base game powerfully recontextualizes the original narrative\nRebuilt RPG elements make combat scenarios truly satisfying regardless of playstyle\nGigs offer fun combat encounters and compelling decisions to make\nRemarkable detail and density give Dogtown a brutal, complex identity\n\nThe Bad\nInconsistent tone with some dialogue options and related character arcs', metadata={'source': 'cyberpunk_2077_phantom_liberty/cyberpunk_2077_phantom_liberty_02.txt'}), Document(page_content="Cyberpunk 2077: Phantom Liberty Review - The Songbird Sings\n\nPhantom Liberty embodies the best of Cyberpunk 2077 for a thrilling RPG-shooter with an evocative story, compelling side content, and unforgettable conclusions.\n\nThe appeal of Night City was in its impressive scope and dazzling visual

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQAWithSourcesChain

prompt_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question. 

{summaries}

Question: {question}

If you don't know the answer, just output a json with all values in the json as 'NA'. Do NOT try to make up an answer.
Only output the JSON. Do NOT output other text.'''

my_question = \
'''Extract the following aspects of the game from the reviews. Output a json with each of the aspects as key, and the extracted information as the value. The format of the json is {"ASPECT":"INFORMATION"}. The aspects are: [Gameplay, Graphics, Sound, Performance, Bug, Suggestion, Price, Overall]
'''

chain02 = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=prompt_template,
            input_variables=["summaries", "question"],        # keep using both input variables.
        )
    },
    return_source_documents=True)

llm_response02 = chain02.invoke({'question':my_question, 'summaries':my_question})

In [3]:
print(llm_response02)

{'question': 'Extract the following aspects of the game from the reviews. Output a json with each of the aspects as key, and the extracted information as the value. The format of the json is {"ASPECT":"INFORMATION"}. The aspects are: [Gameplay, Graphics, Sound, Performance, Bug, Suggestion, Price, Overall]\n', 'summaries': 'Extract the following aspects of the game from the reviews. Output a json with each of the aspects as key, and the extracted information as the value. The format of the json is {"ASPECT":"INFORMATION"}. The aspects are: [Gameplay, Graphics, Sound, Performance, Bug, Suggestion, Price, Overall]\n', 'answer': '{\n"Gameplay": "Expertly written story with two stunning yet wildly different routes and conclusions, great performances make new and returning characters the heart of Cyberpunk, skill tree conversion trimmed for RPG, stealth Bond, John Wick, or Mr. Robot – each game type feels unique",\n"Graphics": "Graphically stunning – Cyberpunk 2077 is the new Crysis",\n"Sou

In [4]:
print(llm_response02['answer'])

{
"Gameplay": "Expertly written story with two stunning yet wildly different routes and conclusions, great performances make new and returning characters the heart of Cyberpunk, skill tree conversion trimmed for RPG, stealth Bond, John Wick, or Mr. Robot – each game type feels unique",
"Graphics": "Graphically stunning – Cyberpunk 2077 is the new Crysis",
"Sound": "World-class acting, not only by Idris Elba and Keanu Reeves",
"Performance": "Rebuilt RPG elements make combat scenarios truly satisfying regardless of playstyle",
"Bug": "NA",
"Suggestion": "Phantom Liberty embodies the best of Cyberpunk 2077 for a thrilling RPG-shooter with an evocative story, compelling side content, and unforgettable conclusions.",
"Price": "NA",
"Overall": "CD Projekt Red’s masterpiece. Not only is Cyberpunk 2077 Phantom Liberty graphically easily three generations ahead of the entire industry and redefines how we experience video games with pathtracing, it’s also written even more thrillingly and stage

---

Without using `RetrievalQA.from_chain_type` or `RetrievalQAWithSourcesChain.from_chain_type`

In [41]:
from pathlib import Path
from langchain_community.llms import Ollama
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader                                                                                                                                                                                                                                                                                                                                                                                                                                      
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain_core.prompts import PromptTemplate

file_dir_path = Path('cyberpunk_2077_phantom_liberty')

llm = Ollama(model="llama2:7b-chat-q4_K_M")
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

loader = DirectoryLoader(str(file_dir_path), glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()                                                              

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# create the vector database
vector_db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_function,
)

retriever = vector_db.as_retriever(search_kwargs={"k": 5})

aspects = ['Gameplay', 'Graphics', 'Sound', 'Performance', 'Bug', 'Suggestion', 'Price', 'Overall']
prompt_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question. 

\'\'\'
{context}
\'\'\'

Question: {question}

If you don't know the answer, just output a json with all values in the json as 'NA'. Do NOT try to make up an answer.
Only output the JSON. Do NOT output other text.'''

my_question = \
'''Extract the following aspects of the game from the reviews. Output a json with each of the aspects as key, and the extracted information as the value. The format of the json is {"ASPECT":"INFORMATION"}. The aspects are: ''' + str(aspects)


query_string = "What are the aspects: " + str(aspects) + " of the game?"
docs = retriever.get_relevant_documents(query=query_string)

prompt_t = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
)
chain = prompt_t | llm


# full example
llm_response = chain.invoke({'question':my_question, 'context': '\n'.join([doc.page_content for doc in docs])})

# print the response
print(llm_response)

# cleanup
vector_db.delete_collection()

Created a chunk of size 1087, which is longer than the specified 1000
Created a chunk of size 1208, which is longer than the specified 1000


Here is the extracted information for each aspect of the game:

{
"Gameplay": {
"Thrilling main story with two stunning yet wildly different routes and conclusions": "True",
"Great performances make new and returning characters the heart of Cyberpunk": "True",
"Rebuilt RPG elements make combat scenarios truly satisfying regardless of playstyle": "True",
"Gigs offer fun combat encounters and compelling decisions to make": "True",
"Remarkable detail and density give Dogtown a brutal, complex identity": "True"
},
"Graphics": {
"Inconsistent tone with some dialogue options and related character arcs": "NA",
"Exceptional visuals that push the boundaries of what's possible on current-gen consoles": "NA"
},
"Sound": {
"Outstanding soundtrack that perfectly complements the game's atmosphere and themes": "True",
"Impressive voice acting that brings the characters to life": "True"
},
"Performance": {
"Solid performance with minimal lag or stuttering": "True",
"Occasional framerate drops or techn

In [42]:
docs

[Document(page_content='The Good\nThrilling main story with two stunning yet wildly different routes and conclusions\nGreat performances make new and returning characters the heart of Cyberpunk\nNew ending for the base game powerfully recontextualizes the original narrative\nRebuilt RPG elements make combat scenarios truly satisfying regardless of playstyle\nGigs offer fun combat encounters and compelling decisions to make\nRemarkable detail and density give Dogtown a brutal, complex identity\n\nThe Bad\nInconsistent tone with some dialogue options and related character arcs', metadata={'source': 'cyberpunk_2077_phantom_liberty/cyberpunk_2077_phantom_liberty_02.txt'}),
 Document(page_content="For all these changes, Cyberpunk 2077 is a vastly better RPG as a result. I felt more ownership over my V than I ever have - and thanks to an ease of access to respec, I was free to experiment and find a play-style that bought me the most glee (in my case, an incredible hacker who fries brains fro

In [43]:
query_string

"What are the aspects: ['Gameplay', 'Graphics', 'Sound', 'Performance', 'Bug', 'Suggestion', 'Price', 'Overall'] of the game?"